In [3]:
import langchain

In [3]:
import dotenv
import os
dotenv.load_dotenv("../.env")

True

In [27]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import TavilySearchResults
from langchain_tavily import TavilySearch
from langchain.prompts import ChatPromptTemplate
import os

# === 1. Initialize the Gemini-based LLM ===
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",  # or "gemini-2.5-flash" for lower latency & cost
    temperature=0.0,
    google_api_key=os.environ["GOOGLE_API_KEY"]
)



# 2. Set up search tool (requires TAVILY_API_KEY in your env)
search = TavilySearch(k=5, tavily_api_key=os.environ["TAVILY_API_KEY"])

# 3. Define prompt
prompt_template = ChatPromptTemplate.from_template("""
You are an AI business analyst.  
Given the following raw web search results about the company "{company}",  
summarize important details for a potential business partner.  
Highlight:
- Core business activities  
- Financial health (if mentioned)  
- Reputation and recent news  
- Any red flags (lawsuits, fraud, layoffs, controversies)  

Results:
{results}

Return a concise, bullet-point summary.
""")

# 4. Function to analyze a company
def analyze_company(company_name):
    search_results = search.run(company_name + " company news financial red flags")
    prompt = prompt_template.format(company=company_name, results=search_results)
    summary = llm.invoke(prompt)
    return summary.content

# Example usage
if __name__ == "__main__":
    print(analyze_company("WeWork"))

Here's a summary of important details about WeWork for a potential business partner:

*   **Core Business Activities:** WeWork primarily provides co-working spaces.
*   **Financial Health:**
    *   Considered a major red flag due to its financial health.
    *   In 2018, despite $1.8 billion in revenue, costs were exceptionally high.
    *   Characterized by increasing leverage and an inflated valuation.
*   **Reputation and Recent News:**
    *   Widely criticized and viewed as a "cautionary tale" in the startup world.
    *   Its business model has been questioned for a lack of innovation.
    *   Associated with significant controversy, including a "cult" narrative around its former leadership.
*   **Red Flags:**
    *   **Severe Financial Concerns:** High costs, increasing debt, and an overvalued assessment.
    *   **Questionable Business Model:** Perceived lack of innovation.
    *   **Insider Selling:** Indicated by reports ahead of its IPO.
    *   **Strong Warnings:** Financi

In [7]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools import TavilySearchResults
import os

# --- 2. Define state ---
class CompanyState(TypedDict):
    company: str
    search_results: str
    summary: str
    red_flags: str
    report: str

# --- 3. Initialize LLM & search ---
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    google_api_key=os.environ["GOOGLE_API_KEY"]
)

search_tool = TavilySearchResults(k=5)

# --- 4. Prompts ---
summarize_prompt = ChatPromptTemplate.from_template("""
You are a business intelligence analyst.
Summarize search results for "{company}" covering:
- Core business activities
- Financial health
- Recent news or events
Results:
{results}
""")

redflag_prompt = ChatPromptTemplate.from_template("""
From this summary, identify any business risks:
- Lawsuits
- Fraud
- Layoffs
- Bankruptcy
If none found, say: "No significant red flags identified."
Summary:
{summary}
""")

# --- 5. Nodes ---
def search_node(state: CompanyState) -> CompanyState:
    results = search_tool.run(f"{state['company']} news financial lawsuits controversies")
    return {**state, "search_results": results}

def summarize_node(state: CompanyState) -> CompanyState:
    prompt = summarize_prompt.format(company=state["company"], results=state["search_results"])
    res = llm.invoke(prompt)
    return {**state, "summary": res.content}

def redflag_node(state: CompanyState) -> CompanyState:
    prompt = redflag_prompt.format(summary=state["summary"])
    res = llm.invoke(prompt)
    return {**state, "red_flags": res.content}

def report_node(state: CompanyState) -> CompanyState:
    report = (
        f"--- Company Intelligence Report ---\n\n"
        f"Company: {state['company']}\n\n"
        f"Summary:\n{state['summary']}\n\n"
        f"Red Flags:\n{state['red_flags']}"
    )
    return {**state, "report": report}

# --- 6. Build graph ---
workflow = StateGraph(CompanyState)

workflow.add_node("search", search_node)
workflow.add_node("summarize", summarize_node)
workflow.add_node("red_flags", redflag_node)
workflow.add_node("report", report_node)

workflow.add_edge(START, "search")
workflow.add_edge("search", "summarize")
workflow.add_edge("summarize", "red_flags")
workflow.add_edge("red_flags", "report")
workflow.add_edge("report", END)

app = workflow.compile()

# --- 7. Run ---
if __name__ == "__main__":
    result = app.invoke({"company": "WeWork"})
    print(result["report"])


--- Company Intelligence Report ---

Company: WeWork

Summary:
Okay, here's a summary of WeWork based on the provided search results, covering its core business, financial health, and recent news:

**Core Business Activities:**

*   WeWork's core business revolves around leasing office buildings and transforming them into shared workspaces. They then rent out this space to freelancers, startups, and larger corporations, often on flexible, short-term leases. The company provides amenities like free beer to tenants.

**Financial Health:**

*   **Pre-Bankruptcy Issues:** WeWork's business model was known to be expensive with a questionable path to profitability. The company was significantly overextended in terms of building leases and renovation costs.
*   **2019 IPO Failure:** WeWork's attempt to go public in 2019 failed spectacularly due to concerns about its financials, business practices, and the leadership of its CEO, Adam Neumann. This led to major funders pulling out and the compa

In [8]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [9]:
from langchain_tavily import TavilySearch
from langgraph.prebuilt import create_react_agent

# Initialize Tavily Search Tool
tavily_search_tool = TavilySearch(
    max_results=5,
    topic="general",
)

agent = create_react_agent(llm, [tavily_search_tool])

user_input = "What are potential red flag about company called OpenAi as a potential business partner"

for step in agent.stream(
    {"messages": user_input},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What are potential red flag about company called OpenAi as a potential business partner
================================== Ai Message ==================================
Tool Calls:
  tavily_search (4efe7d50-76d7-45d1-af95-0ec9e562101a)
 Call ID: 4efe7d50-76d7-45d1-af95-0ec9e562101a
  Args:
    query: OpenAI red flags business partner OR concerns OpenAI partnership
================================= Tool Message =================================
Name: tavily_search

{"query": "OpenAI red flags business partner OR concerns OpenAI partnership", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.linkedin.com/posts/allisonlcarter_the-scoop-deepseek-raises-red-flags-for-activity-7290401488625942528-sD6i", "title": "DeepSeek raises red flags for OpenAI, U.S. Navy - PR Daily | Allison ...", "content": "I caution anyone using this for their business not to share any propriet